In [58]:
# Data loading

import pandas as pd
import numpy as np
import os

data = pd.read_csv(os.path.join('data', 'X_train.csv'))
test_data = pd.read_csv(os.path.join('data', 'X_test.csv'))
labels = pd.read_csv(os.path.join('data', 'y_train.csv'), header=None)

In [59]:
data.isna().sum(), test_data.isna().sum()

(tBodyAcc-mean()-X                       3667
 tBodyAcc-mean()-Y                       3656
 tBodyAcc-mean()-Z                       3632
 tBodyAcc-std()-X                        1476
 tBodyAcc-std()-Y                        3668
                                         ... 
 angle(tBodyGyroMean,gravityMean)        3802
 angle(tBodyGyroJerkMean,gravityMean)    3638
 angle(X,gravityMean)                    1482
 angle(Y,gravityMean)                    3629
 angle(Z,gravityMean)                    1505
 Length: 561, dtype: int64,
 tBodyAcc-mean()-X                       1434
 tBodyAcc-mean()-Y                       1412
 tBodyAcc-mean()-Z                       1482
 tBodyAcc-std()-X                         568
 tBodyAcc-std()-Y                        1471
                                         ... 
 angle(tBodyGyroMean,gravityMean)        1492
 angle(tBodyGyroJerkMean,gravityMean)    1474
 angle(X,gravityMean)                     582
 angle(Y,gravityMean)                    1512
 angle

In [60]:
# A lot of nans, let's fill it with zeros for now
test_data.fillna(0, inplace=True)
data.fillna(0, inplace=True)

In [61]:
# split data
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25)

In [62]:
# train and score simple logreg model
model = LogisticRegression(max_iter=1e5)
model.fit(X_train, y_train.values.ravel())
model.score(X_test, y_test)

0.6142546245919478

In [63]:
# not so good, so there is plenty room of improvement
# e.g you can use timeseries provided
# lets train on full data and submit
model = LogisticRegression(max_iter=1e5)
model.fit(data, labels.values.ravel())
submission = pd.DataFrame({'id': test_data.index, 'Category': model.predict(test_data)})
submission.to_csv(os.path.join('kaggle_output', 'sample_submission2.csv'), index=None)

In [ ]:
# 62.5%, neat
# GL : )